<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/tools/DistillationNRTL_EtOH_iPrOH_H2O_che4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che4.ipynb -O che4.ipynb
!pip install importnb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
from importnb import Notebook
with Notebook(): 
    from che4 import DotDict, Range, RangeArray, Comp, CompArray, d2nt, Props

import numpy as np
from scipy.optimize import root,minimize
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True)

In [7]:
p=Props(['Ethanol','Isopropanol', 'Water'])

In [167]:
d=DotDict()

d.P = 1e5
d.N = 15
d.NC = p.N_comps
d.F= 100.
d.Fz = np.array([0.1,0.1, 0.8])
d.FT =320.
d.D = 0.15*d.F
d.B = d.F - d.D

d.F_stage = d.N // 2

d.reflux_ratio = 10


In [168]:
def boiling_points(P):
    return root(lambda T: np.diagonal(p.Pvap(T)) - P, p.Tb).x

In [169]:
d.bp = boiling_points(d.P)
bp_scaled = (d.bp - np.mean(d.bp))/1.2 + np.mean(d.bp)
bp_scaled_minmax = np.min(bp_scaled), np.max(bp_scaled)
bp_minmax = np.min(d.bp), np.max(d.bp)

d.T = RangeArray(np.linspace(*bp_scaled_minmax,d.N), *bp_minmax)
d.TC = Range(bp_scaled[-1], *bp_minmax)

Lguess = np.repeat(d.D*(d.reflux_ratio),d.N)
Lguess[d.F_stage:]+=d.F

d.L = RangeArray(Lguess, 0., 2*d.F*d.reflux_ratio)
d.V = RangeArray(np.repeat(d.D*(d.reflux_ratio+1),d.N), 0., 2*d.F*d.reflux_ratio)

QBGuess = d.D * (d.reflux_ratio+1)* np.mean(p.HvapNB)
d.QB = Range(QBGuess, 0., 2*QBGuess )
d.QC = Range(QBGuess, 0., 2*QBGuess)

d.Lx = CompArray(np.tile(d.Fz,(d.N,1)))
d.Vy = CompArray(np.tile(d.Fz,(d.N,1)))



In [170]:
x0, x2nt, wrap, x2unk, const   = d2nt(d)

In [239]:
r = DotDict()

r.MB = np.zeros((d.N,d.NC))
r.EB = np.zeros(d.N)
zeros = np.zeros((1,p.N_comps))
def eqs(d):
    r.EQUIL = d.Lx * p.NRTL_gamma(d.Lx,d.T)* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = np.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = np.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = np.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB[:] = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB[d.F_stage-1] += d.F*d.Fz

    r.TCBP = np.atleast_1d(np.sum(d.Vy[0]* p.NRTL_gamma(d.Vy[0],d.TC) * p.Pvap(d.TC)) - d.P)

    r.T = np.insert(d.T,0,d.TC)
    r.EB[:] = p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:])
    r.EB[d.F_stage-1] += p.Hl(d.F*d.Fz, d.FT)
    r.EB[-1]+=d.QB

    r.CONDENSER = np.atleast_1d(p.Hv(r.Vout[0],r.T[1]) - p.Hl(r.Vout[0], r.T[0]) - d.QC)

    r.RF = np.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return np.concatenate([np.ravel(r.EQUIL), np.ravel(r.MB), np.ravel(r.EB), r.TCBP, r.RF, r.CONDENSER]), r


In [240]:

def eqs_jax(d):
    zeros = jnp.zeros((1,p.N_comps))
    r = DotDict()
    r.EQUIL = d.Lx * p.NRTL_gamma(d.Lx,d.T)* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = jnp.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = jnp.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = jnp.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB=r.MB.at[d.F_stage-1].add(d.F*d.Fz)

    r.TCBP = jnp.atleast_1d(jnp.sum(d.Vy[0]* p.NRTL_gamma(d.Vy[0],d.TC) * p.Pvap(d.TC)) - d.P)

    r.T = jnp.insert(d.T,0,d.TC)
    r.EB = p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:])
    r.EB=r.EB.at[d.F_stage-1].add(p.Hl(d.F*d.Fz, d.FT))
    r.EB=r.EB.at[-1].add(d.QB)

    r.CONDENSER = jnp.atleast_1d(p.Hv(r.Vout[0],r.T[1]) - p.Hl(r.Vout[0], r.T[0]) - d.QC)

    r.RF = jnp.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return jnp.concatenate([jnp.ravel(r.EQUIL), jnp.ravel(r.MB), jnp.ravel(r.EB), r.TCBP, r.RF, r.CONDENSER]), r

In [248]:
eqs_wrapped = wrap(eqs_jax)
eqs_jit=jax.jit(eqs_wrapped)
jac=jax.jit(jax.jacobian(eqs_wrapped))

In [262]:
sol=minimize(lambda x: 0., x0, method='SLSQP', bounds=[(-15,15)]*len(x0), constraints=dict(type='eq',fun=eqs_jit,jac=jac))
# sol=root(eqs_jit,x0,jac=jac,method='lm')

In [261]:
eqs_jax(x2nt(sol.x))

(DeviceArray([-1.38243195e-10, -2.18278728e-10,  1.67347025e-10,
              -7.27595761e-12, -8.00355338e-11,  7.27595761e-12,
               1.97542249e-09,  2.12457962e-09,  1.69166015e-09,
               5.24960342e-09,  7.42147677e-09,  5.71526471e-09,
              -3.63797881e-10,  4.07453626e-10,  4.36557457e-11,
               8.00355338e-11,  3.63797881e-10, -8.73114914e-11,
              -2.29192665e-10, -6.47560228e-10, -1.45519152e-10,
              -8.36735126e-11,  1.30967237e-10, -1.60071068e-10,
              -6.54836185e-11,  1.96450856e-10, -8.00355338e-11,
              -1.23691279e-10, -1.23691279e-10, -2.69210432e-10,
              -2.91038305e-11,  1.23691279e-10,  1.16415322e-10,
               3.63797881e-12, -8.00355338e-11, -5.09317033e-11,
               1.05501385e-10,  5.67524694e-10,  3.34694050e-10,
               5.45696821e-11, -1.45519152e-10, -1.38243195e-10,
               4.00177669e-11,  6.18456397e-11, -1.38243195e-10,
              -7.10542736

In [265]:
x2nt(sol.x)._asdict()

OrderedDict([('P', DeviceArray(100000., dtype=float64)),
             ('N', DeviceArray(15, dtype=int64)),
             ('NC', DeviceArray(3, dtype=int64)),
             ('F', DeviceArray(100., dtype=float64)),
             ('Fz', DeviceArray([0.1, 0.1, 0.8], dtype=float64)),
             ('FT', DeviceArray(320., dtype=float64)),
             ('D', DeviceArray(15., dtype=float64)),
             ('B', DeviceArray(85., dtype=float64)),
             ('F_stage', DeviceArray(7, dtype=int64)),
             ('reflux_ratio', DeviceArray(10, dtype=int64)),
             ('bp',
              DeviceArray([351.12430999, 355.285959  , 372.77897168], dtype=float64)),
             ('T',
              DeviceArray([352.44013578, 352.48097275, 352.5219882 , 352.57101141,
                           352.64880986, 352.82555878, 353.46267848, 353.46182256,
                           353.46094446, 353.460276  , 353.46107074, 353.47002109,
                           353.52399687, 353.86826911, 357.32306792], d